Team members:

Name: Xi Yu
StudentID: 22015116

Name: Chenxin Li
Student ID: 22066519

The second line in the book should be a comment stating if the code executes to the end without an error.

The third line in the book should be a comment with a link to the original source where you opted to reuse an existing implementation.


try cnn here, refer to https://www.kaggle.com/code/mdmosarrofhossen/brain-tumor-detection-accuracy-100-val-acc-100

kaggle dataset here: https://www.kaggle.com/datasets/navoneel/brain-mri-images-for-brain-tumor-detection

maybe not enough large. We will try to train and see the process time.

Add GPU to accelerate

In [ ]:
import time
start_time = time.time()

In [ ]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print(device_name)
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))
tf.device('/device:GPU:0')

Found GPU at: /device:GPU:0


In [ ]:
!rm -rf Brain_MRI_Image
! git clone https://github.com/TravellerXi/Brain_MRI_Image

Cloning into 'Brain_MRI_Image'...
remote: Enumerating objects: 3031, done.
remote: Counting objects: 100% (242/242), done.
remote: Compressing objects: 100% (238/238), done.
remote: Total 3031 (delta 4), reused 242 (delta 4), pack-reused 2789
Receiving objects: 100% (3031/3031), 78.64 MiB | 11.11 MiB/s, done.
Resolving deltas: 100% (4/4), done.
Updating files: 100% (3128/3128), done.


In [ ]:
! ls

Brain_MRI_Image  sample_data


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
#import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator

import warnings
warnings.filterwarnings('ignore')

np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
#intializing the image size and batch size
height = 244
width = 244
batch_size=32

In [ ]:
dataset_train_dir = "Brain_MRI_Image/train/"
dataset_test_dir = "Brain_MRI_Image/test/"

In [ ]:
datagen_test = ImageDataGenerator(
            rescale=1./255.,
            )

test_ds=datagen_test.flow_from_directory(
        dataset_test_dir,
        shuffle=True,
        batch_size=32,
        class_mode="binary",
        target_size=(height, width),
        classes={'no': 0., 'yes': 1.}
    )
def image_gen(height, width):
    datagen_train = ImageDataGenerator(
            rescale=1./255.,
            validation_split=0.2,
            )
    train_ds = datagen_train.flow_from_directory(
        dataset_train_dir,
        batch_size=batch_size,
        subset="training",
        shuffle=True,
        class_mode="binary",
        target_size=(height, width),
        classes={'no': 0., 'yes': 1.}
    )
    
    val_ds = datagen_train.flow_from_directory(
        dataset_train_dir,
        batch_size=batch_size,
        subset="validation",
        shuffle=True,
        class_mode="binary",
        target_size=(height, width),
        classes={'no': 0., 'yes': 1.}
    )

   
    return train_ds, val_ds
    

#train_ds, val_ds, test_ds = image_gen(height, width)
train_ds, val_ds = image_gen(height, width)

Found 38 images belonging to 2 classes.
Found 2469 images belonging to 2 classes.
Found 616 images belonging to 2 classes.


In [ ]:
(X_train,y_train) = next(train_ds)
(X_val,y_val) = next(val_ds)

In [ ]:
# Merge inputs and targets
inputs = np.concatenate((X_train, X_val), axis=0)
targets = np.concatenate((y_train, y_val), axis=0)

In [ ]:
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.losses import sparse_categorical_crossentropy
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import KFold
import numpy as np
from keras.applications import ResNet50V2
from keras.layers import Dense, Flatten, GlobalAveragePooling2D, Dropout, GlobalMaxPooling2D
from keras.optimizers import Adam
from keras.models import Sequential, load_model

In [ ]:
# Define the K-fold Cross Validator
acc_per_fold=[]
loss_per_fold=[]
num_folds=10
kfold = KFold(n_splits=num_folds, shuffle=True)

# K-fold Cross Validation model evaluation
fold_no = 1
for train, val in kfold.split(inputs, targets):

  # Define the model architecture
  model = ResNet50V2(input_shape=(244,244,3), include_top=False,pooling="avg")

  # Compile the model
  model.compile(optimizer=Adam(), 
               loss="sparse_categorical_crossentropy", 
               metrics=['accuracy'])


  # Generate a print
  print('------------------------------------------------------------------------')
  print(f'Training for fold {fold_no} ...')
  checkpoint = tf.keras.callbacks.ModelCheckpoint('model/resnet50v2_best.h5', 
                                                monitor='accuracy', verbose=1, 
                                                mode='max',save_best_only=True)
  early = tf.keras.callbacks.EarlyStopping(monitor="accuracy", mode="max",restore_best_weights=True, patience=5)
  callbacks_list = [checkpoint,early]


  # Fit data to model
  history = model.fit(inputs[train], targets[train], epochs=30, callbacks=callbacks_list)

  # Generate generalization metrics
  scores = model.evaluate(inputs[val], targets[val], verbose=0)
  print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
  acc_per_fold.append(scores[1] * 100)
  loss_per_fold.append(scores[0])

  # Increase fold number
  fold_no = fold_no + 1

------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/30
2/2 [==============================] - ETA: 0s - loss: 8.5293 - accuracy: 0.0000e+00
Epoch 1: accuracy improved from -inf to 0.00000, saving model to model/resnet50v2_best.h5
2/2 [==============================] - 10s 4s/step - loss: 8.5293 - accuracy: 0.0000e+00
Epoch 2/30
2/2 [==============================] - ETA: 0s - loss: 6.4445 - accuracy: 0.0000e+00
Epoch 2: accuracy did not improve from 0.00000
2/2 [==============================] - 1s 265ms/step - loss: 6.4445 - accuracy: 0.0000e+00
Epoch 3/30
2/2 [==============================] - ETA: 0s - loss: 3.6762 - accuracy: 0.1930
Epoch 3: accuracy improved from 0.00000 to 0.19298, saving model to model/resnet50v2_best.h5
2/2 [==============================] - 2s 2s/step - loss: 3.6762 - accuracy: 0.1930
Epoch 4/30
2/2 [==============================] - ETA: 0s - loss: 0.9067 - accuracy: 0.7544
Epoch 4: accuracy improved from 0

Score for fold 5: loss of 18.89985466003418; accuracy of 0.0%
------------------------------------------------------------------------
Training for fold 6 ...
Epoch 1/30
2/2 [==============================] - ETA: 0s - loss: 9.8201 - accuracy: 0.0000e+00 
Epoch 1: accuracy improved from -inf to 0.00000, saving model to model/resnet50v2_best.h5
2/2 [==============================] - 8s 2s/step - loss: 9.8201 - accuracy: 0.0000e+00
Epoch 2/30
2/2 [==============================] - ETA: 0s - loss: 6.4647 - accuracy: 0.0000e+00
Epoch 2: accuracy did not improve from 0.00000
2/2 [==============================] - 1s 284ms/step - loss: 6.4647 - accuracy: 0.0000e+00
Epoch 3/30
2/2 [==============================] - ETA: 0s - loss: 4.4989 - accuracy: 0.0172    
Epoch 3: accuracy improved from 0.00000 to 0.01724, saving model to model/resnet50v2_best.h5
2/2 [==============================] - 2s 2s/step - loss: 4.4989 - accuracy: 0.0172
Epoch 4/30
2/2 [==============================] - ETA: 0s -

Score for fold 6: loss of 18.55394172668457; accuracy of 0.0%
------------------------------------------------------------------------
Training for fold 7 ...
Epoch 1/30
2/2 [==============================] - ETA: 0s - loss: 8.7028 - accuracy: 0.0000e+00
Epoch 1: accuracy improved from -inf to 0.00000, saving model to model/resnet50v2_best.h5
2/2 [==============================] - 7s 2s/step - loss: 8.7028 - accuracy: 0.0000e+00
Epoch 2/30
2/2 [==============================] - ETA: 0s - loss: 5.7949 - accuracy: 0.0000e+00
Epoch 2: accuracy did not improve from 0.00000
2/2 [==============================] - 1s 280ms/step - loss: 5.7949 - accuracy: 0.0000e+00
Epoch 3/30
2/2 [==============================] - ETA: 0s - loss: 3.2030 - accuracy: 0.2414
Epoch 3: accuracy improved from 0.00000 to 0.24138, saving model to model/resnet50v2_best.h5
2/2 [==============================] - 2s 2s/step - loss: 3.2030 - accuracy: 0.2414
Epoch 4/30
2/2 [==============================] - ETA: 0s - loss

In [ ]:
print(acc_per_fold)
print(loss_per_fold)

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[4.9840874671936035, 19.137405395507812, 5.572181701660156, 18.449127197265625, 18.89985466003418, 18.55394172668457, 17.22808265686035, 18.335737228393555, 1.75102961063385, 9.060660362243652]
